In [24]:
# SELFACT_CBC_OTHERIND.IPYNB - A simple case of resource compeition CBC applied to a self-activating gene with another inducer level, with a basic model
# By Kirill Sechkar

# PACKAGE IMPORTS
import numpy as np
import pickle
import jax
import jax.numpy as jnp
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
import matplotlib as mpl, matplotlib.pyplot as plt
from bokeh import io as bkio
from bokeh.colors import RGB as bkRGB

# SOLVER AND CONTROLLER IMPORTS
import common.controllers as ctrls
import common.reference_switchers as refsws
import common.ode_solvers as odesols

# BIFURCATION ANALYSIS, PROBE CHARACTERISATION AND PREDICTION TOOLS
from common import selfact_an_bif as an_tools
from common.jointexp import *
from common.probe_char_tools import *

# CELL AND CIRCUIT MODEL IMPORTS
from basic_model import *
import basic_genetic_modules as gms

# set up jax
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)

# set up bokeh
bkio.reset_output()
bkplot.output_notebook()

# set up matplotlib
%matplotlib widget

Loading BokehJS ...

In [25]:
# INITIALISE THE MODEL, ITS PARAMETERS AND INITIAL CONDITIONS
model_auxil = ModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = model_auxil.default_params()  # get default parameter values
init_conds = model_auxil.default_init_conds(par)  # get default initial conditions

In [26]:
# DEFINE CIRCUIT PARAMETERS TO BE CONSIDERED
circ_par={}

circ_par['q_s'] = 120.0  # resource competition factor for the switch gene
circ_par['q_ofp'] = 100*circ_par['q_s'] # RC factor for the switch's fluorescent output gene
circ_par['F_s,0'] = 0.05
circ_par['K_s'] = 250.0
circ_par['I_s'] = 1/9

circ_par['q_ta'] = 45.0  # RC factor for the transcription activation factor
circ_par['q_b'] = 6e4  # RC factor for the burdensome gene of the probe
circ_par['mu_b'] = np.log(2) / (13.6 / 60)
circ_par['F_b,0'] = 0.01

In [27]:
# INITIALISE AND PARAMETERISE THE CELL MODEL

# initialise cell model
model_auxil = ModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
model_par = model_auxil.default_params()  # get default parameter values
init_conds = model_auxil.default_init_conds(model_par)  # get default initial conditions

# add reference tracker switcher
model_par_with_refswitch, ref_switcher = model_auxil.add_reference_switcher(model_par,
                                                                                    # cell model parameters
                                                                                    refsws.no_switching_initialise,
                                                                                    # function initialising the reference switcher
                                                                                    refsws.no_switching_switch
                                                                                    # function switching the references to be tracked
                                                                                    )

# load synthetic genetic modules and the controller
odeuus_complete, \
    module1_F_calc, module2_F_calc, \
    module1_specterms, module2_specterms, \
    controller_action, controller_update, \
    par, init_conds, controller_memo0, \
    synth_genes_total_and_each, synth_miscs_total_and_each, \
    controller_memos, controller_dynvars, controller_ctrledvar, \
    modules_name2pos, modules_styles, controller_name2pos, controller_styles, \
    module1_v_with_F_calc, module2_v_with_F_calc = model_auxil.add_modules_and_controller(
        # module 1
        gms.sas_initialise,  # function initialising the circuit
        gms.sas_ode,  # function defining the circuit ODEs
        gms.sas_F_calc, # function calculating the circuit genes' transcription regulation functions
        gms.sas_specterms, # function calculating the circuit genes effective mRNA levels (due to possible co-expression from the same operons)
        # module 2
        gms.cicc_initialise,  # function initialising the circuit
        gms.cicc_ode,  # function defining the circuit ODEs
        gms.cicc_F_calc, # function calculating the circuit genes' transcription regulation functions
        gms.cicc_specterms, # function calculating the circuit genes effective mRNA levels (due to possible co-expression from the same operons)
        # controller
        ctrls.cci_initialise,  # function initialising the controller
        ctrls.cci_action,  # function calculating the controller action
        ctrls.cci_ode,  # function defining the controller ODEs
        ctrls.cci_update,  # function updating the controller based on measurements
        # cell model parameters and initial conditions
        model_par_with_refswitch, init_conds)

# unpack the synthetic genes and miscellaneous species lists
synth_genes = synth_genes_total_and_each[0]
module1_genes = synth_genes_total_and_each[1]
module2_genes = synth_genes_total_and_each[2]
synth_miscs = synth_miscs_total_and_each[0]
module1_miscs = synth_miscs_total_and_each[1]
module2_miscs = synth_miscs_total_and_each[2]

par.update(circ_par)

In [28]:
# GET MAXIMUM BURDENS AND DEGRADATION COEFFICIENTS

q_r_and_q_o = par['q_r'] + par['q_o']

# calculate maximum synthetic gene burdens
Qmaxs = {
    'Q_s_max': par['q_s'] / q_r_and_q_o,
    'Q_ofp_max': par['q_ofp'] / q_r_and_q_o,
    'Q_ta_max': par['q_ta'] / q_r_and_q_o,
    'Q_b_max': par['q_b'] / q_r_and_q_o,
}

# calculate native gene burdens
Qs_native = {
    'Q_r': par['q_r'] / q_r_and_q_o,
    'Q_o': par['q_o'] / q_r_and_q_o,
}

# no degradation => zero chis
chis={'chi_s':0.0, 'chi_ofp':0.0}

# record maximum burdens and degradation coefficients in a common dictionary
cellvars={}
cellvars.update(Qmaxs)
cellvars.update(Qs_native)
cellvars.update(chis)
# add maxmium overall burden from the self-activating switch
cellvars['Q_sas_max']=cellvars['Q_s_max']+cellvars['Q_ofp_max']
# add translation elongation rate
cellvars['e']=par['e']

# set zero 'housekeeping protein mass fraction' parameter, just to reuse the cell model tools
par['phi_q'] = 0.0

In [29]:
# GET THE TWO BIFURCATION POINTS

# for one of them, F_req touches the F_real curve from below; for the other, from above
bif_Freqbelow, bif_Freqabove = an_tools.find_bifurcations(par, cellvars)

In [30]:
# GET THE ANALYTICAL BIFURCATION CURVE

# define the range of probe-exerted burdens to consider
Q_probe_range=np.linspace(0.0, 0.5, 30)

# get the bifurcation curve
an_bif_curve = an_tools.find_equilibria_for_Q_osynth_range(Q_probe_range,
                                                           bif_Freqbelow, bif_Freqabove,
                                                           par, cellvars)

In [31]:
# PLOT THE ANALYTICAL BIFURCATION CURVE

# p_ofp vs imposed burden
an_ofp_fig = bkplot.figure(
    frame_width=360,
    frame_height=360,
    x_axis_label="Q_p, probe burden",
    y_axis_label="ofp_mature, mature output fluorescent protein level",
    x_range=(min(Q_probe_range), max(Q_probe_range)),
    y_range=(0, max(an_bif_curve['ofp_mature'])),
    title='Analytical bifurcation curve: p_ofp vs imposed burden',
    tools="box_zoom,pan,hover,reset"
)
# plot the controlled variable vs control action
an_ofp_fig.line(x=an_bif_curve['Q_osynth'],
             y=an_bif_curve['ofp_mature'],
             line_width=1.5, line_color='black', line_dash='dashed',
             legend_label='true steady states')
an_ofp_fig.scatter(x=an_bif_curve['Q_osynth'],
                y=an_bif_curve['ofp_mature'],
                marker='circle', size=5,
                color='black', legend_label='true steady states')
# legend formatting
an_ofp_fig.legend.label_text_font_size = "8pt"
an_ofp_fig.legend.location = "top_right"
an_ofp_fig.legend.click_policy = 'hide'

# exreted burden vs imposed burden
an_Q_fig = bkplot.figure(
    frame_width=360,
    frame_height=360,
    x_axis_label="Q_p, probe burden",
    y_axis_label="Q_sas, self-activating switch burden",
    x_range=(0, 0.5),
    y_range=(0, 0.5),
    title='Analytical bifurcation curve: exerted vs imposed burden',
    tools="box_zoom,pan,hover,reset"
)
# plot the controlled variable vs control action
an_Q_fig.line(x=an_bif_curve['Q_osynth'],
             y=an_bif_curve['Q_sas'],
             line_width=1.5, line_color='black', line_dash='dashed',
             legend_label='true steady states')
an_Q_fig.scatter(x=an_bif_curve['Q_osynth'],
                y=an_bif_curve['Q_sas'],
                marker='circle', size=5,
                color='black', legend_label='true steady states')
# legend formatting
an_Q_fig.legend.label_text_font_size = "8pt"
an_Q_fig.legend.location = "top_right"
an_Q_fig.legend.click_policy = 'hide'

# show the plots
bkplot.show(bklayouts.grid([[an_ofp_fig, an_Q_fig]]))

In [32]:
# SIMULATE CBC - INITIALISE AND PARAMETERISE THE CELL MODEL

# add reference tracker switcher
model_par_with_refswitch, ref_switcher = model_auxil.add_reference_switcher(model_par,
                                                                                    # cell model parameters
                                                                                    refsws.timed_switching_initialise,
                                                                                    # function initialising the reference switcher
                                                                                    refsws.timed_switching_switch
                                                                                    # function switching the references to be tracked
                                                                                    )

# load synthetic genetic modules and the controller
odeuus_complete, \
    module1_F_calc, module2_F_calc, \
    module1_specterms, module2_specterms, \
    controller_action, controller_update, \
    par, init_conds, controller_memo0, \
    synth_genes_total_and_each, synth_miscs_total_and_each, \
    controller_memos, controller_dynvars, controller_ctrledvar, \
    modules_name2pos, modules_styles, controller_name2pos, controller_styles, \
    module1_v_with_F_calc, module2_v_with_F_calc = model_auxil.add_modules_and_controller(
    # module 1
    gms.sas_initialise,  # function initialising the circuit
    gms.sas_ode,  # function defining the circuit ODEs
    gms.sas_F_calc, # function calculating the circuit genes' transcription regulation functions
    gms.sas_specterms, # function calculating the circuit genes effective mRNA levels (due to possible co-expression from the same operons)
    # module 2
    gms.cicc_initialise,  # function initialising the circuit
    gms.cicc_ode,  # function defining the circuit ODEs
    gms.cicc_F_calc,    # function calculating the circuit genes' transcription regulation functions
    gms.cicc_specterms,  # function calculating the circuit genes effective mRNA levels (due to possible co-expression from the same operons)
    # controller
    ctrls.pichem_initialise,  # function initialising the controller
    ctrls.pichem_action,  # function calculating the controller action
    ctrls.pichem_ode,  # function defining the controller ODEs
    ctrls.pichem_update,  # function updating the controller based on measurements
    # cell model parameters and initial conditions
    model_par_with_refswitch, init_conds)

# unpack the synthetic genes and miscellaneous species lists
synth_genes = synth_genes_total_and_each[0]
module1_genes = synth_genes_total_and_each[1]
module2_genes = synth_genes_total_and_each[2]
synth_miscs = synth_miscs_total_and_each[0]
module1_miscs = synth_miscs_total_and_each[1]
module2_miscs = synth_miscs_total_and_each[2]

par.update(circ_par)

In [33]:
# SIMULATE CBC - SET CONTROLLER AND SIMULATION PARAMETER

controller_ctrledvar='ofp_mature'

# reference p_ofp levels
p_s_sup = an_tools.find_p_s_sup(par, cellvars)    # maximum p_switch
_,ofp_mature_sup = an_tools.p_ofp_immat_mat(p_s_sup, 0.0, par, cellvars)
cbc_refs=np.linspace(0, ofp_mature_sup, 30)
cbc_refs=np.flip(cbc_refs)

# simulation parameters

# switching between references
par['t_burn_in']=24.0
cbc_exp_duration=240.0
par['t_switch_ref']=(cbc_exp_duration/len(cbc_refs))

tf = (0.0, par['t_burn_in']+cbc_exp_duration)  # simulation time
meastimestep = 0.1  # hours


# PI controller parameters - gains negative as more probe induction => less p_ofp
par['Kp'] = -0.0005
# par['Kp'] = -0.001
par['Ki'] = 0.0

# initial control action
u0=0.0

control_delay=0.01

In [34]:
# SIMULATE CBC - RUN THE SIMULATION

# set up the ODE solver
ode_solver, us_size = odesols.create_euler_solver(odeuus_complete,
                                                  control_delay=control_delay,
                                                  meastimestep=meastimestep,
                                                  euler_timestep=1e-5)

# solve ODE
timer= time.time()
ts_jnp, xs_jnp,\
    ctrl_memorecord_jnp, uexprecord_jnp, \
    refrecord_jnp  = ode_sim(par,   # model parameters
                             ode_solver,    # ODE solver for the cell with the synthetic gene circuit
                             odeuus_complete,    # ODE function for the cell with the synthetic gene circuit and the controller (also gives calculated and experienced control actions)
                             controller_ctrledvar,    # name of the variable read and steered by the controller
                             controller_update, controller_action,   # function for updating the controller memory and calculating the control action
                             model_auxil.x0_from_init_conds(init_conds,
                                                                par,
                                                                synth_genes, synth_miscs, controller_dynvars,
                                                                modules_name2pos,
                                                                controller_name2pos),   # initial condition VECTOR
                             controller_memo0,  # initial controller memory record
                             u0,    # initial control action, applied before any measurement-informed actions reach the system
                             (len(synth_genes), len(module1_genes), len(module2_genes)),    # number of synthetic genes
                             (len(synth_miscs), len(module1_miscs), len(module2_miscs)),    # number of miscellaneous species
                             modules_name2pos, controller_name2pos, # dictionaries mapping gene names to their positions in the state vector
                             model_auxil.synth_gene_params_for_jax(par, synth_genes),   # synthetic gene parameters in jax.array form
                             tf, meastimestep,  # simulation time frame and measurement time step
                             control_delay,  # delay before control action reaches the system
                             us_size,  # size of the control action record needed
                             cbc_refs, ref_switcher,  # reference values and reference switcher
                             )

# convert simulation results to numpy arrays
ts = np.array(ts_jnp)
xs = np.array(xs_jnp)
ctrl_memorecord = np.array(ctrl_memorecord_jnp)
uexprecord = np.array(uexprecord_jnp)
refrecord= np.array(refrecord_jnp)

In [35]:
# PLOT - SYNTHETIC CIRCUITS AND CONTROLLER
# plot synthetic circuit concentrations
mRNA_fig, prot_fig, misc_fig = model_auxil.plot_circuit_concentrations(ts, xs,
                                                                          par, synth_genes, synth_miscs,
                                                                          modules_name2pos,
                                                                          modules_styles)  # plot simulation results
# plot synthetic circuit regulation functions
F_fig = model_auxil.plot_circuit_regulation(ts, xs, # time points and state vectors
                                                ctrl_memorecord, uexprecord,    # controller memory and experienced control actions records
                                                refrecord,  # reference tracker records
                                                module1_F_calc, module2_F_calc, # transcription regulation functions for both modules
                                                controller_action, # control action calculator
                                                par, # model parameters
                                                synth_genes_total_and_each,     # list of synthetic genes - total and for each module
                                                synth_miscs_total_and_each,     # list of synthetic miscellaneous species - total and for each module
                                                modules_name2pos,   # dictionary mapping gene names to their positions in the state vector
                                                module1_specterms, module2_specterms, # functions calculating effective mRNA levels for both modules
                                                controller_name2pos, # dictionary mapping controller species to their positions in the state vector
                                                modules_styles)  # plot simulation results
# plot controller memory, dynamic variables and actions
ctrl_ref_fig, ctrl_memo_fig, ctrl_dynvar_fig, ctrl_u_fig = model_auxil.plot_controller(ts, xs,
                                                                                        ctrl_memorecord, uexprecord, # controller memory and experienced control actions records
                                                                                        refrecord, # reference tracker records
                                                                                        controller_memos, controller_dynvars,
                                                                                        controller_ctrledvar,
                                                                                        controller_action, controller_update,
                                                                                        par,
                                                                                        synth_genes, synth_miscs,
                                                                                        modules_name2pos,
                                                                                        module1_specterms, module2_specterms,
                                                                                        controller_name2pos,
                                                                                        controller_styles,
                                                                                        u0, control_delay)
bkplot.show(bklayouts.grid([[mRNA_fig, prot_fig, misc_fig],
                            [F_fig, None, ctrl_ref_fig],
                            [ctrl_memo_fig, ctrl_dynvar_fig, ctrl_u_fig]]))

In [36]:
# GET THE CALCULATED CONTROL ACTIONS FOR THE CBC TRAJECTORY

# get the calculated control actions
ucalcrecord = model_auxil.get_u_calc(ts, xs, ctrl_memorecord, refrecord,
                                         controller_action,
                                         par,
                                         synth_genes, synth_miscs,
                                         modules_name2pos,
                                         module1_specterms, module2_specterms,
                                         controller_name2pos,
                                         controller_ctrledvar)

In [37]:
# GET THE (REAL) BURDENS EXPERIENCED AND IMPOSED BY THE SELF-ACTIVATING SWITCH

Fs_sas, Fs_probe = model_auxil.get_Fs(ts, xs,  # time points and state vectors
                                          uexprecord,  # experienced control actions records
                                          module1_F_calc, module2_F_calc, # transcription regulation functions for both modules
                                          par,  # model parameters
                                          synth_genes_total_and_each,   # list of synthetic genes - total and for each module
                                          module1_specterms, module2_specterms, # functions calculating effective mRNA levels for both modules
                                          modules_name2pos # dictionary mapping gene names to their positions in the state vector
                                          )

# get the burdens experienced by the self-activating switch
Q_probes=Fs_probe[:, modules_name2pos['F_ta']]*cellvars['Q_ta_max'] + Fs_probe[:, modules_name2pos['F_b']]*cellvars['Q_b_max']

# get the burdens imposed by the self-activating switch
Q_sass=Fs_sas[:, modules_name2pos['F_s']]*cellvars['Q_sas_max']

In [38]:
# PROCESS THE CBC TRAJECTORY

# gradient relative tolerance for steady state
grad_rtol=1

# go through the trajectory and find average p_ofp in the last hour of tracking each reference
ucalc_steady_states=[]
uexp_steady_states=[]
ofp_steady_states=[]
b_steady_states=[]
Q_probe_steady_states=[]
Q_sas_steady_states=[]
ref_cntr=0
for i in range(1, len(ts)):
    if((refrecord[i]!=refrecord[i-1]) or (i==len(ts)-1)):
        # find the time points for the last hour of tracking
        last_hour_indices=np.where((ts>=ts[i-1]-1) & (ts<=ts[i-1]))[0]

        # get the mean u and p_ofp in the last hour of tracking
        ucalc_mean=np.mean(ucalcrecord[last_hour_indices])
        uexp_mean=np.mean(uexprecord[last_hour_indices])
        ofp_mean=np.mean(xs[last_hour_indices, modules_name2pos[controller_ctrledvar]])
        b_mean=np.mean(xs[last_hour_indices, modules_name2pos['b_mature']])
        # real burdens
        Q_probe_mean=np.mean(Q_probes[last_hour_indices])
        Q_sas_mean=np.mean(Q_sass[last_hour_indices])

        # before recording the value, check if the reference is somewhat steady
        grad, _ = np.polyfit(ts[last_hour_indices],
                             xs[last_hour_indices,modules_name2pos[controller_ctrledvar]],
                             deg=1)  # get a linear fit for controlled variable vs time
        # if check is passed, record the steady state
        if(abs(grad/ofp_mean)<grad_rtol):
            ucalc_steady_states.append(ucalc_mean)
            uexp_steady_states.append(uexp_mean)
            ofp_steady_states.append(ofp_mean)
            b_steady_states.append(b_mean)
            # real burdens
            Q_probe_steady_states.append(Q_probe_mean)
            Q_sas_steady_states.append(Q_sas_mean)

        # move to the next reference
        ref_cntr+=1

In [39]:
# GET THE ESTIMATED BURDENS EXPERIENCED AND IMPOSED BY THE SELF-ACTIVATING SWITCH

# load data from the probe characterisation experiment
Q_probe_data = np.load('basic_Q_probe_data.npy')
Qdash_probe_data = np.load('basic_Qdash_probe_data.npy')

# create interpolation function (comment out the unwanted onptions)
# linear ND option
Q_probe_interpolator = make_interpolator_sp_lnd(Q_probe_data,
                                          normalise_u_and_y=True)
Q_sas_interpolator = make_interpolator_sp_lnd(Qdash_probe_data,
                                              normalise_u_and_y=True)

# get the estimated burdens
Q_probe_steady_states_est=np.array(Q_probe_steady_states)
Q_sas_steady_states_est=np.zeros_like(Q_sas_steady_states)
for i in range(0, len(uexp_steady_states)):
    Q_probe_steady_states_est[i]=Q_probe_interpolator(ucalc_steady_states[i],b_steady_states[i])
    Q_sas_steady_states_est[i]=Q_sas_interpolator(ucalc_steady_states[i],b_steady_states[i])

In [40]:
# PLOT BIFURCATION DIAGRAM

# u values displayed on the plot
u_range=(0.01,11)


# for convenience of display, zero action shown as minimal action
ucalcrecord[ucalcrecord==0]=min(u_range)
uexprecord[uexprecord==0]=min(u_range)

# plot the cbc trajectory as actually observed
cbc_fig = bkplot.figure(
    frame_width=480,
    frame_height=360,
    x_axis_type="log",
    x_axis_label="u, probe induction",
    y_axis_label="ofp_mature, controlled variable",
    x_range=(min(u_range), max(u_range)),
    y_range=(0, max(an_bif_curve['ofp_mature'])),
    title='Control action vs controlled variable',
    tools="box_zoom,pan,hover,reset,save"
)
cbc_fig.output_backend = 'svg'
# plot the cbc trajectory for calculated control actions
cbc_fig.line(x=uexprecord, y=xs[:, modules_name2pos['ofp_mature']], line_width=1.5,
             line_color='blue', line_alpha=0.5,
             legend_label='cbc trajectory (real)')

# plot the CBC steady states
cbc_fig.scatter(x=uexp_steady_states, y=ofp_steady_states,
                marker='circle', size=5,
               color='violet', legend_label='cbc steady states (real)')
# plot the estimated CBC steady states
cbc_fig.scatter(x=ucalc_steady_states, y=ofp_steady_states,
                marker='circle', size=5,
               color='darkviolet', legend_label='cbc steady states (est)')

# legend formatting
cbc_fig.legend.label_text_font_size = "8pt"
cbc_fig.legend.location = "bottom_left"
cbc_fig.legend.click_policy = 'hide'

# plot the cbc trajectory for imposed and exerted burdens
cbc_Q_fig = bkplot.figure(
    frame_width=480,
    frame_height=360,
    x_axis_label="Q_p, probe's resource demand",
    y_axis_label="Q_sas, self-activating switch's demand",
    x_range=(0, 0.75),
    y_range=(0, 0.25),
    title='Burden plot',
    tools="box_zoom,pan,hover,reset,save"
)
cbc_Q_fig.output_backend = 'svg'
# plot the analytical bifurcation curve
cbc_Q_fig.line(x=an_bif_curve['Q_osynth'],
             y=an_bif_curve['Q_sas'],
             line_width=1.5, line_color='black', line_dash='dashed',
             legend_label='true steady states')
cbc_Q_fig.scatter(x=an_bif_curve['Q_osynth'],
                y=an_bif_curve['Q_sas'],
                marker='circle', size=5,
                color='black', legend_label='true steady states')

# plot the cbc trajectory for imposed and exerted burdens
cbc_Q_fig.line(x=Q_probes, y=Q_sass,
               line_width=1.5, line_color='blue', line_alpha=0.5,
               legend_label='cbc trajectory (real)')

# plot the CBC steady states
cbc_Q_fig.scatter(x=Q_probe_steady_states, y=Q_sas_steady_states,
                marker='circle', size=5,
               color='violet', legend_label='cbc steady states (real)')
# plot the estimated CBC steady states
cbc_Q_fig.scatter(x=Q_probe_steady_states, y=Q_sas_steady_states_est,
                marker='circle', size=5,
               color='darkviolet', legend_label='cbc steady states (est)')

# legend formatting
cbc_Q_fig.legend.label_text_font_size = "8pt"
cbc_Q_fig.legend.location = "top_right"
cbc_Q_fig.legend.click_policy = 'hide'


# show plot
bkplot.show(bklayouts.grid([[cbc_fig, cbc_Q_fig]]))

In [41]:
# GET BIFURCATION CURVES STEADY STATES FOR TWO SELF-ACTIVATING SWITCHES EXPRESSED IN THE SAME CELL

# change notation for clarity
# bifurcation curve for the first switch
file = open("selfact_bif.pkl","rb")
bif_loaded=pickle.load(file)
file.close()
Q_sas1s_1 = bif_loaded['Q_moi']    # resource competition imposed by the first switch
Q_sas2s_1 = bif_loaded['Qdash_moi']   # resource competition experienced by the first switch - Q_probe in CBC experiment, Q_sas1 for joint expression
ofp1s_1 = bif_loaded['ofp_steady'] # outputs corresponding to the CBC bifurcation curve points
# bifurcation curve for the second switch - FROM THE RESULTS HERE
Q_sas2s_2 = Q_sas_steady_states_est     # resource competition imposed by the first switch
Q_sas1s_2 = Q_probe_steady_states_est   # resource competition experienced by the first switch - Q_probe in CBC experiment, Q_sas2 for joint expression
ofp2s_2 = ofp_steady_states # outputs corresponding to the CBC bifurcation curve points

# estimate outputs of the competing switch for each bifurcation curve, based on the other bifurcation curve
# bifurcation curve for the first switch
ofp2s_1 = np.zeros(len(ofp1s_1)-2)
for i in range(1,len(ofp1s_1)-1):
    ofp2s_1[i-1] = Q_sas_to_ofp_mature(Q_sas_query=Q_sas2s_1[i],
                                     Q_sass_bifcurve=Q_sas2s_2,
                                     ofps_bifcurve=ofp2s_2
                                     )
# bifurcation curve for the second switch
ofp1s_2 = np.zeros(len(ofp2s_2)-2)
for i in range(1,len(ofp2s_2)-1):
    ofp1s_2[i-1] = Q_sas_to_ofp_mature(Q_sas_query=Q_sas1s_2[i],
                                  Q_sass_bifcurve=Q_sas1s_1,
                                  ofps_bifcurve=ofp1s_1,
                                  )

In [42]:
# FIND STEADY STATES YIELDED BY CURVE INTERSECTIONS
# (work with resource competition factors, then just predict outputs by interpolation)

# consider all possible pairs of two curves' segments, find intersections
Q_sas1s_intersect, Q_sas2s_intersect = bifcurves_intersection(Q_sas1s_1,
                                                              Q_sas2s_1,
                                                              Q_sas1s_2,
                                                              Q_sas2s_2)

# find the corresponding switch outputs
ofp1s_intersect = Q_sas_to_ofp_mature(Q_sas_query=Q_sas1s_intersect,
                                      Q_sass_bifcurve=Q_sas1s_1,
                                      ofps_bifcurve=ofp1s_1,
                                      )
ofp2s_intersect = Q_sas_to_ofp_mature(Q_sas_query=Q_sas2s_intersect,
                                      Q_sass_bifcurve=Q_sas2s_2,
                                      ofps_bifcurve=ofp2s_2,
                                      )

In [43]:
# SIMULATE JOINT EXPRESSION - INITIALISE AND PARAMETERISE THE CELL MODEL

# initialise cell model
jointexp_model_par = model_auxil.default_params()  # get default parameter values
jointexp_init_conds = model_auxil.default_init_conds(jointexp_model_par)  # get default initial conditions

# add reference tracker switcher
jointexp_model_par_with_refswitch, jointexp_ref_switcher = model_auxil.add_reference_switcher(jointexp_model_par,
                                                                                    # cell model parameters
                                                                                    refsws.no_switching_initialise,
                                                                                    # function initialising the reference switcher
                                                                                    refsws.no_switching_switch
                                                                                    # function switching the references to be tracked
                                                                                    )

# load synthetic genetic modules and the controller
jointexp_odeuus_complete, \
    jointexp_module1_F_calc, jointexp_module2_F_calc, \
    jointexp_module1_specterms, jointexp_module2_specterms, \
    jointexp_controller_action, jointexp_controller_update, \
    jointexp_par, jointexp_init_conds, jointexp_controller_memo0, \
    jointexp_synth_genes_total_and_each, jointexp_synth_miscs_total_and_each, \
    jointexp_controller_memos, jointexp_controller_dynvars, jointexp_controller_ctrledvar, \
    jointexp_modules_name2pos, jointexp_modules_styles, jointexp_controller_name2pos, jointexp_controller_styles, \
    jointexp_module1_v_with_F_calc, jointexp_module2_v_with_F_calc = model_auxil.add_modules_and_controller(
    # module 1
    gms.sas_initialise,  # function initialising the circuit
    gms.sas_ode,  # function defining the circuit ODEs
    gms.sas_F_calc,  # function calculating the circuit genes' transcription regulation functions
    gms.sas_specterms,
    # function calculating the circuit genes effective mRNA levels (due to possible co-expression from the same operons)
    # module 2
    gms.sas2_initialise,  # function initialising the circuit
    gms.sas2_ode,  # function defining the circuit ODEs
    gms.sas2_F_calc,  # function calculating the circuit genes' transcription regulation functions
    gms.sas2_specterms,
    # function calculating the circuit genes effective mRNA levels (due to possible co-expression from the same operons)
    # controller
    ctrls.cci_initialise,  # function initialising the controller
    ctrls.cci_action,  # function calculating the controller action
    ctrls.cci_ode,  # function defining the controller ODEs
    ctrls.cci_update,  # function updating the controller based on measurements
    # cell model parameters and initial conditions
    jointexp_model_par_with_refswitch, jointexp_init_conds)

# unpack the synthetic genes and miscellaneous species lists
jointexp_synth_genes = jointexp_synth_genes_total_and_each[0]
jointexp_module1_genes = jointexp_synth_genes_total_and_each[1]
jointexp_module2_genes = jointexp_synth_genes_total_and_each[2]
jointexp_synth_miscs = jointexp_synth_miscs_total_and_each[0]
jointexp_module1_miscs = jointexp_synth_miscs_total_and_each[1]
jointexp_module2_miscs = jointexp_synth_miscs_total_and_each[2]

# update switch parameters
jointexp_circ_par=circ_par.copy()
for sas1_par in circ_par.keys():
    gene_for_par=sas1_par.split('_')[-1]
    if(gene_for_par=='s' or gene_for_par=='ofp'):
        jointexp_circ_par[sas1_par+'2']=circ_par[sas1_par]   # system assumed to be symmetric
    elif(sas1_par=='eta_s'):
        jointexp_circ_par['eta_s2,s2']=circ_par[sas1_par] 
    elif(sas1_par=='K_s'):
        jointexp_circ_par['K_s2,s2']=circ_par[sas1_par]
    elif(sas1_par=='F_s,0'):
        jointexp_circ_par['F_s2,0']=circ_par[sas1_par]
jointexp_par.update(jointexp_circ_par)

# set up the ODE solver
jointexp_tf=(0.0,48.0)
jointexp_control_delay=0.0
jointexp_ode_solver, jointexp_us_size = odesols.create_euler_solver(jointexp_odeuus_complete,
                                                  control_delay=jointexp_control_delay,
                                                  meastimestep=meastimestep,
                                                  euler_timestep=1e-5)
jointexp_controller_ctrledvar='ofp_mature'

In [44]:
# SIMULATE JOINT EXPRESSION - GET THE STATE WITHOUT ANY INDUCTION

# get starting steady state without induction
jointexp_par['I_s']=0.0
jointexp_par['I_s2']=0.0
ts_jnp_ss0, xs_jnp_ss0,\
    _, _, \
    _  = ode_sim(jointexp_par,   # model parameters
                 jointexp_ode_solver,    # ODE solver for the cell with the synthetic gene circuit
                 jointexp_odeuus_complete,    # ODE function for the cell with the synthetic gene circuit and the controller (also gives calculated and experienced control actions)
                 jointexp_controller_ctrledvar,    # name of the variable read and steered by the controller
                 jointexp_controller_update, jointexp_controller_action,   # function for updating the controller memory and calculating the control action
                 model_auxil.x0_from_init_conds(jointexp_init_conds,
                                                    jointexp_par,
                                                    jointexp_synth_genes, jointexp_synth_miscs, jointexp_controller_dynvars,
                                                    jointexp_modules_name2pos,
                                                    jointexp_controller_name2pos),   # initial condition VECTOR
                 jointexp_controller_memo0,  # initial controller memory record
                 0.0,    # initial control action, applied before any measurement-informed actions reach the system - irrelevant here
                 (len(jointexp_synth_genes), len(jointexp_module1_genes), len(jointexp_module2_genes)),    # number of synthetic genes
                 (len(jointexp_synth_miscs), len(jointexp_module1_miscs), len(jointexp_module2_miscs)),    # number of miscellaneous species
                 jointexp_modules_name2pos, jointexp_controller_name2pos, # dictionaries mapping gene names to their positions in the state vector
                 model_auxil.synth_gene_params_for_jax(jointexp_par, jointexp_synth_genes),   # synthetic gene parameters in jax.array form
                 jointexp_tf, meastimestep,  # simulation time frame and measurement time step
                 jointexp_control_delay,  # delay before control action reaches the system
                 jointexp_us_size,  # size of the control action record needed
                 np.array([0.0]), jointexp_ref_switcher,  # reference values and reference switcher
                 )
jointexp_x0_noind=np.array(xs_jnp_ss0[-1,:])

In [45]:
# SIMULATE JOINT EXPRESSION - RUN FOR DIFERENT INITIAL CONDITIONS
jointexp_p_ofps=[]
jointexp_p_ofp2s=[]
for i in range(0,3):
    jointexp_x0=jointexp_x0_noind.copy()
    if(i==0):
        jointexp_x0[jointexp_modules_name2pos['p_s']]+=jointexp_x0_noind[jointexp_modules_name2pos['p_s2']]
        jointexp_x0[jointexp_modules_name2pos['p_s2']]=0.0
        jointexp_x0[jointexp_modules_name2pos['p_ofp']]+=jointexp_x0_noind[jointexp_modules_name2pos['p_ofp2']]
        jointexp_x0[jointexp_modules_name2pos['p_ofp2']]=0.0
        jointexp_x0[jointexp_modules_name2pos['ofp_mature']]+=jointexp_x0_noind[jointexp_modules_name2pos['ofp2_mature']]
        jointexp_x0[jointexp_modules_name2pos['ofp2_mature']]=0.0
    elif(i==1):
        jointexp_x0[jointexp_modules_name2pos['p_s2']]+=jointexp_x0_noind[jointexp_modules_name2pos['p_s']]
        jointexp_x0[jointexp_modules_name2pos['p_s']]=0.0
        jointexp_x0[jointexp_modules_name2pos['p_ofp2']]+=jointexp_x0_noind[jointexp_modules_name2pos['p_ofp']]
        jointexp_x0[jointexp_modules_name2pos['p_ofp']]=0.0
        jointexp_x0[jointexp_modules_name2pos['ofp2_mature']]+=jointexp_x0_noind[jointexp_modules_name2pos['ofp_mature']]
        jointexp_x0[jointexp_modules_name2pos['ofp_mature']]=0.0
    # turn on induction
    jointexp_par['I_s']=bif_loaded['I_s']# FROM THE LOADED BIFURCATION CURVE
    jointexp_par['I_s2']=jointexp_circ_par['I_s'] # FROM HERE

    # simulate
    ts_jnp, xs_jnp,\
    _, _, \
    _  = ode_sim(jointexp_par,   # model parameters
                 jointexp_ode_solver,    # ODE solver for the cell with the synthetic gene circuit
                 jointexp_odeuus_complete,    # ODE function for the cell with the synthetic gene circuit and the controller (also gives calculated and experienced control actions)
                 jointexp_controller_ctrledvar,    # name of the variable read and steered by the controller
                 jointexp_controller_update, jointexp_controller_action,   # function for updating the controller memory and calculating the control action
                 jointexp_x0,   # initial condition VECTOR
                 jointexp_controller_memo0,  # initial controller memory record
                 0.0,    # initial control action, applied before any measurement-informed actions reach the system - irrelevant here
                 (len(jointexp_synth_genes), len(jointexp_module1_genes), len(jointexp_module2_genes)),    # number of synthetic genes
                 (len(jointexp_synth_miscs), len(jointexp_module1_miscs), len(jointexp_module2_miscs)),    # number of miscellaneous species
                 jointexp_modules_name2pos, jointexp_controller_name2pos, # dictionaries mapping gene names to their positions in the state vector
                 model_auxil.synth_gene_params_for_jax(jointexp_par, jointexp_synth_genes),   # synthetic gene parameters in jax.array form
                 jointexp_tf, meastimestep,  # simulation time frame and measurement time step
                 jointexp_control_delay,  # delay before control action reaches the system
                 jointexp_us_size,  # size of the control action record needed
                 np.array([0.0]), jointexp_ref_switcher,  # reference values and reference switcher
                 )

    # convert simulation results to numpy arrays
    jointexp_p_ofps.append(np.array(xs_jnp[:, jointexp_modules_name2pos['ofp_mature']]))
    jointexp_p_ofp2s.append(np.array(xs_jnp[:, jointexp_modules_name2pos['ofp2_mature']]))

In [46]:
# CDC PAPER FIGURE 5C
# plot the resource demand bifurcation curves of both self-activating switches, identify their intersections
predict_c_fig = bkplot.figure(
    frame_width=75,
    frame_height=75,
    x_axis_label="Q\'moi1,t=Qmoi2,t",
    y_axis_label="Qmoi1,t=Q\'moi2,t",
    x_range=(0, 0.45),
    y_range=(0, 0.45),
    tools="box_zoom,pan,hover,reset,save"
)
predict_c_fig.output_backend = 'svg'

# plot the estimated CBC steady states for the first switch
predict_c_fig.line(x=Q_sas2s_1, y=Q_sas1s_1,
                    line_width=1.5, color='#bb3385ff',
                    legend_label='MOI1 (I1=1/10)')
# plot the estimated CBC steady states for the second switch
predict_c_fig.line(x=Q_sas2s_2, y=Q_sas1s_2,
                    line_width=1.5, color='#ff6700f8',
                    legend_label='MOI2 (I2=1/9)')

# mark the overall steady states
predict_c_fig.scatter(x=Q_sas2s_intersect, y=Q_sas1s_intersect,
                         marker='circle', size=5,
                         fill_alpha=0.0,
                         line_color='#6495edff',
                         line_width=2)

# legend formatting
predict_c_fig.legend.location = "top_right"
predict_c_fig.legend.label_text_font_size = "6pt"
predict_c_fig.legend.padding=0
predict_c_fig.legend.margin=2
predict_c_fig.legend.spacing=0
predict_c_fig.legend.glyph_width=5
predict_c_fig.legend.click_policy = 'hide'

# axis formatting
predict_c_fig.xaxis.ticker=bkmodels.FixedTicker(ticks=[0, 0.15, 0.3, 0.45])
predict_c_fig.yaxis.ticker=bkmodels.FixedTicker(ticks=[0, 0.15, 0.3, 0.45])
predict_c_fig.xaxis.axis_label_text_font_size = "6pt"
predict_c_fig.yaxis.axis_label_text_font_size = "6pt"
predict_c_fig.xaxis.axis_label_text_color = "black"
predict_c_fig.yaxis.axis_label_text_color = "black"
predict_c_fig.xaxis.major_label_text_font_size = "6pt"
predict_c_fig.yaxis.major_label_text_font_size = "6pt"
predict_c_fig.xaxis.major_label_text_color = "black"
predict_c_fig.yaxis.major_label_text_color = "black"

# CDC PAPER FIGURE 5D: PREDICTING STEADY-STATE OUTPUTS
predict_d_fig = bkplot.figure(
    frame_width=75,
    frame_height=75,
    x_axis_label="MOI2 output [nM]",
    y_axis_label="MOI1 output [nM]",
    x_range=(0, 4.5e5),
    y_range=(0, 4.5e5),
    tools="box_zoom,pan,hover,reset,save"
)
predict_d_fig.output_backend = 'svg'

# plot the simulates trjaectories
for i in range(0,3):
    predict_d_fig.line(x=jointexp_p_ofp2s[i], y=jointexp_p_ofps[i],
                                 line_width=2, line_color='gray', line_alpha=0.5,
                                 legend_label='sim. trajs.')

# plot the identified steady states
predict_d_fig.scatter(x=ofp2s_intersect, y=ofp1s_intersect,
                      marker='circle', size=5,
                      fill_alpha=0.0,
                      line_color='#6495edff',
                      line_width=2,
                      legend_label='pred. steady\nstates')

# legend formatting
predict_d_fig.legend.location = "top_right"
predict_d_fig.legend.label_text_font_size = "6pt"
predict_d_fig.legend.padding=0
predict_d_fig.legend.margin=2
predict_d_fig.legend.spacing=0
predict_d_fig.legend.glyph_width=10
predict_d_fig.legend.click_policy = 'hide'
predict_d_fig.legend.orientation = "horizontal"

# axis formatting
predict_d_fig.xaxis.formatter = bkmodels.PrintfTickFormatter(format="%4.1e")
predict_d_fig.yaxis.formatter = bkmodels.PrintfTickFormatter(format="%4.1e")
predict_d_fig.xaxis.ticker=bkmodels.FixedTicker(ticks=[0, 1.5e5, 3e5, 4.5e5])
predict_d_fig.yaxis.ticker=bkmodels.FixedTicker(ticks=[0, 1.5e5, 3e5, 4.5e5])
predict_d_fig.xaxis.axis_label_text_font_size = "6pt"
predict_d_fig.yaxis.axis_label_text_font_size = "6pt"
predict_d_fig.xaxis.axis_label_text_color = "black"
predict_d_fig.yaxis.axis_label_text_color = "black"
predict_d_fig.xaxis.major_label_text_font_size = "6pt"
predict_d_fig.yaxis.major_label_text_font_size = "6pt"
predict_d_fig.xaxis.major_label_text_color = "black"
predict_d_fig.yaxis.major_label_text_color = "black"

# show plots
bkplot.show(bklayouts.grid([[predict_c_fig, predict_d_fig]]))